In [1]:
%matplotlib notebook
import time
import sklearn
import pandas as pd
import numpy as np
import category_encoders
from numpy.linalg.linalg import LinAlgError
from collections import Counter
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression as lr
from sklearn.naive_bayes import BernoulliNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix as confu
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.cluster import KMeans
from itertools import combinations as Cb
from sklearn.tree import ExtraTreeClassifier as ex_tree


/usr/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [14]:
# Normalize data
def normalization(training_numpy, testing_numpy):
    min_max = preprocessing.MinMaxScaler()
    min_max.fit(training_numpy)
    x_scaled = min_max.fit_transform(training_numpy)
    testing_x_scaled = min_max.transform(testing_numpy)
    #training_norm = pd.DataFrame(x_scaled, columns = columns)
    #testing_norm = pd.DataFrame(testting_x_scaled, columns = columns)
    return (x_scaled, testing_x_scaled)

def pca_function(rate, data):
    pca = PCA()
    pca.fit(data)
    for thres_n in xrange(1,len(data)):
        if sum(pca.explained_variance_ratio_[:thres_n])>rate:
            pca_n = thres_n
            break
    
    pca = PCA(n_components=pca_n)
    pca_data = pca.fit_transform(data)
    return (pca_data, pca)

def lda_whole_function(X, y, n_cluster=2):
    
    original_y = y.copy()
    anomaly_index = []
    
    
    for uni_label in np.unique(y):
        anomaly_index.append(np.where(y == uni_label)[0])

    #km_anomaly = KMeans(n_clusters=n_cluster)
    km_anomaly_model = []
    anomaly_label = []
    for i in range(len(anomaly_index)):
        km_anomaly = KMeans(n_clusters=n_cluster)
        km_anomaly_model.append(km_anomaly.fit(X.iloc[anomaly_index[i]], y[anomaly_index[i]]))
        anomaly_label.append(km_anomaly.labels_)
        
    #print len(anomaly_label),np.unique(anomaly_label[0])
    for i in range(len(anomaly_index)):
        km_uni_len = len(np.unique(anomaly_label[i]))
        for (j,now_label) in zip(range(n_cluster*(i+1), n_cluster*(i+1)-n_cluster, -1), range(n_cluster-1,-1,-1)):
            #print "i",i,"j:",j, "now_label:", now_label
            anomaly_label[i][np.where(anomaly_label[i] == now_label)[0]] = j
        y[anomaly_index[i]] = anomaly_label[i]
    #print np.unique(y)
    #print len(np.where(normal_label == 0)[0]),len(np.where(normal_label == 1)[0]), len(np.where(anomaly_label == 2)[0]), len(np.where(anomaly_label == 3)[0])
    #print np.unique(y)
    #return (X,y)
    #print np.unique(y)
    #print lda.n_components
    try:
        lda = LDA()
        lda_data = lda.fit_transform(X, y)
    except ValueError:
        lda = LDA()
        lda_data = lda.fit_transform(X, original_y)
    except LinAlgError:
        lda = LDA()
        lda_data = lda.fit_transform(X, original_y)
        
    #print lda_data.shape
    return (lda_data, lda)


def lda_function(X, y, n_cluster=2):
    
    original_y = y.copy()
    anomaly_index = []
    
    
    for uni_label in np.unique(y):
        anomaly_index.append(np.where(y == uni_label)[0])

    #km_anomaly = KMeans(n_clusters=n_cluster)
    km_anomaly_model = []
    anomaly_label = []
    for i in range(len(anomaly_index)):
        km_anomaly = KMeans(n_clusters=n_cluster)
        km_anomaly_model.append(km_anomaly.fit(X.iloc[anomaly_index[i]], y[anomaly_index[i]]))
        anomaly_label.append(km_anomaly.labels_)
        
    #print len(anomaly_label),np.unique(anomaly_label[0])
    for i in range(len(anomaly_index)):
        km_uni_len = len(np.unique(anomaly_label[i]))
        for (j,now_label) in zip(range(n_cluster*(i+1), n_cluster*(i+1)-n_cluster, -1), range(n_cluster-1,-1,-1)):
            #print "i",i,"j:",j, "now_label:", now_label
            anomaly_label[i][np.where(anomaly_label[i] == now_label)[0]] = j
        y[anomaly_index[i]] = anomaly_label[i]
    #print np.unique(y)
    #print len(np.where(normal_label == 0)[0]),len(np.where(normal_label == 1)[0]), len(np.where(anomaly_label == 2)[0]), len(np.where(anomaly_label == 3)[0])
    #print np.unique(y)
    #return (X,y)
    #print np.unique(y)
    
    #print lda.n_components
    try:
        lda = LDA()
        lda_data = lda.fit_transform(X, y)
    except ValueError:
        lda = LDA()
        lda_data = lda.fit_transform(X, original_y)
    except LinAlgError:
        lda = LDA()
        lda_data = lda.fit_transform(X, original_y)
        
    #print lda_data.shape
    return (lda_data, lda)


'''
def pca_function(n, data):
    pca = PCA(n_components=n)
    pca_data = pca.fit_transform(data)
    return (pca_data.flatten(), pca)

def lda_function(X, y):
    lda = LDA()
    lda_data = lda.fit_transform(X, y)
    return (lda_data.flatten(), lda)
'''
def replace_value(training_normal_numpy, key, loc):  
    training_normal_numpy[loc][key] = 1
    
def del_zero_column(df_train, df_test):
    new_df_test = pd.DataFrame(np.zeros([df_test.shape[0], df_train.shape[1]]),columns=df_train.columns)
    new_df_test[list(set(df_train.columns) & set(df_test.columns))] = df_test[list(set(df_train.columns) & set(df_test.columns))]
    #df_test = df_test[df_train.columns]
    return (df_train, new_df_test)

def combine_function(x):
    new_x = ''
    for i in range(len(x)):
        new_x = new_x + str(x[i]) +'_'
    return new_x[:-1]
#def del_zero_column(df_train, df_test):
#    df_train = df_train.loc[:, (df_train != 0).any(axis=0)]
#    df_test = df_test[df_train.columns]
#    return (df_train, df_test)

def combine_function(x):
    new_x = ''
    for i in range(len(x)):
        new_x = new_x + str(x[i]) +'_'
    return new_x[:-1]

def all_lda(train_combine, label_num, n_cluster = 2):
    train_all_dummy = pd.get_dummies(train_combine)
    #train_all_dummy = category_encoders.OneHotEncoder(cols=train_combine.columns.tolist()).fit_transform(train_combine)
    (train_lda, lda) = lda_whole_function(train_all_dummy, np.array(label_num), n_cluster=n_cluster)
    clf=BernoulliNB()
    start = time.time()
    
    score = np.average(cross_val_score(clf, train_lda, label_num, cv=10))
    return (score , time.time()-start)

def divide_lda(train_combine, label_num, n_cluster = 2):
    train_column= train_combine.columns
    for col in train_combine.columns:
        train_dummy.append(pd.get_dummies(train_combine[col]))
        #train_dummy.append(category_encoders.OneHotEncoder(cols=[col]).fit_transform(train_combine[[col]]))
    train_lda = pd.DataFrame(index=range(len(label_num)))
    for i in range(len(train_combine.columns)):
        #print train_dummy[i].shape
        (now_train_lda, lda) = lda_whole_function(train_dummy[i], np.array(label_num), n_cluster=3)
        now_lda_col_name = []
        for j in range(len(now_train_lda[0])):
            train_lda[train_column[i]+'_'+str(j+1)] = np.array(now_train_lda)[:,j]
    start = time.time()
    score = np.average(cross_val_score(clf, train_lda, label_num, cv=10))
    return (score, time.time()-start)

 

# DNA

In [ ]:
# Do not run yet
train=pd.read_csv('splice.data.txt', header=None)
train.columns=['label', 'name', 'dna']
key_str = {}
for (i,label_name) in zip(range(len(np.unique(train['label']))), np.unique(train['label'])):
    key_str[label_name] = i
    
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key == label else None,key_str.keys()), train['label'])
train_dummy = []
    
train['dna'] = train['dna'].map(lambda x: list(str(x).strip()))
for idx in xrange(60):
    train['dna_%d'% (idx,)] = train['dna'].map(lambda x: x[idx])

train_column = train.columns[3:]
train = train[train_column]

result = pd.DataFrame(columns=['combine_num','score', 'time'])



total_score = []
total_time = []
clf = SVC(kernel='linear', C=1.0)

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)

exp_num =0
for combine_num in range(3,5):
    
    train_combine = pd.DataFrame(index=range(len(train)))
    com_index = 0
    for sub in Cb(train.columns,combine_num):
        combine_list = []
        map(lambda data: combine_list.append(combine_function(data)),train[list(sub)].values)
        train_combine['combine_%d' % com_index] = combine_list
        com_index = com_index + 1
    #print "combine_subset:", com_index
    train_dummy = pd.get_dummies(train_combine)

    for iteration in range(1):
        val_time = []
        now_score = []
        for train_val_index, test_val_index in skf:    
            lda = LDA()
            try:    
                train_lda = lda.fit_transform(train_dummy.iloc[train_val_index], val_label[train_val_index])
                test_lda = lda.transform(train_dummy.iloc[test_val_index])
                (train_lda, test_lda) = normalization(train_lda, test_lda)
                clf.fit(train_lda, val_label[train_val_index])

                start = time.time()
                now_score.append(accuracy_score(clf.predict(test_lda), val_label[test_val_index]))
                val_time.append(time.time() - start)     
            except LinAlgError:
                None
        exe_time = np.sum(val_time)
        total_score.append(np.average(now_score))
        total_time.append(exe_time)
            
    print "combine_num", combine_num
    print "score:", np.average(total_score)
    print "time:", np.average(total_time)
    result.loc[exp_num] = [combine_num, np.average(total_score), np.average(total_time)]
    exp_num = exp_num + 1
    result.to_csv('dna_combine_lda_normal(3-4).csv')

In [ ]:
def combine_selection(train_dummy, label):
    columns = train_dummy.columns
    feature_dic = {}
    new_column = []
    initial = 1
    whole_train_dummy = pd.DataFrame()
    for column, idx in zip(columns, range(len(columns))):     
        now_column = []
        each_train_dummy = pd.get_dummies(train_dummy[column])
        
        clf = ex_tree()
        clf.fit(each_train_dummy, label)
        
        for key, value in zip(each_train_dummy.columns, clf.feature_importances_):
            now_column.append(('combine%d_'+key) % idx)
            new_column.append(('combine%d_'+key) % idx)
            feature_dic[('combine%d_'+key) % idx] = value
        each_train_dummy.columns = now_column
        if initial == 1:
            whole_train_dummy = whole_train_dummy.append(each_train_dummy)
            initial = 0
        else:
            whole_train_dummy = whole_train_dummy.join(each_train_dummy)
        
    #print len(new_column)
    sorted_dic = sorted(feature_dic.iteritems(), key=lambda (k,v): (v,k),reverse=True)
    
        
    
    return (sorted_dic, whole_train_dummy) 

In [184]:
def combine_selection(train_dummy, label, test_dummy, threshold):
    columns = train_dummy.columns
    feature_dic = {}
    new_column = []
    initial = 1
    whole_train_dummy = pd.DataFrame()
    start = time.time()
    for column, idx in zip(columns, range(len(columns))):
    
        now_column = []
        each_train_dummy = pd.get_dummies(train_dummy[column])
        
        clf = ex_tree()
        clf.fit(each_train_dummy, label)
        
        for key, value in zip(each_train_dummy.columns, clf.feature_importances_):
            now_column.append(('combine%d_'+key) % idx)
            new_column.append(('combine%d_'+key) % idx)
            feature_dic[('combine%d_'+key) % idx] = value
        each_train_dummy.columns = now_column
        if initial == 1:
            whole_train_dummy = whole_train_dummy.append(each_train_dummy)
            initial = 0
        else:
            whole_train_dummy = whole_train_dummy.join(each_train_dummy)
    print "Information gain process time:", time.time()-start
    start = time.time()
    #print len(new_column)
    sorted_dic = sorted(feature_dic.iteritems(), key=lambda (k,v): (v,k),reverse=True)
    sorted_feature = [i[0] for i in sorted_dic][:threshold]
        
    new_train_dummy = whole_train_dummy[sorted_feature]
    new_test_dummy = pd.DataFrame(np.zeros([test_dummy.shape[0], len(sorted_feature)]),columns=sorted_feature, index=test_dummy.index)
    test_dummy = pd.get_dummies(test_dummy)
    
    cross_column = list(set(sorted_feature) & set(test_dummy.columns))
    new_test_dummy[cross_column] = test_dummy[cross_column]
    print "Feature selection process time:", time.time()-start
    #print sorted_dic
    return (new_train_dummy, new_test_dummy) 

In [ ]:
# Do not run yet
train=pd.read_csv('splice.data.txt', header=None)
train.columns=['label', 'name', 'dna']
key_str = {}
for (i,label_name) in zip(range(len(np.unique(train['label']))), np.unique(train['label'])):
    key_str[label_name] = i
    
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key == label else None,key_str.keys()), train['label'])
train_dummy = []
    
train['dna'] = train['dna'].map(lambda x: list(str(x).strip()))
for idx in xrange(60):
    train['dna_%d'% (idx,)] = train['dna'].map(lambda x: x[idx])

train_column = train.columns[3:]
train = train[train_column]

result = pd.DataFrame(columns=['combine_num','c_cluster','score', 'time'])



total_score = []
total_time = []
clf = SVC(kernel='linear', C=1.0)
#clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)

exp_num = 0
for combine_num in range(2,4):
    
    train_combine = pd.DataFrame(index=range(len(train)))
    com_index = 0
    for sub in Cb(train.columns,combine_num):
        combine_list = []
        map(lambda data: combine_list.append(combine_function(data)),train[list(sub)].values)
        train_combine['combine%d' % com_index] = combine_list
        com_index = com_index + 1
    print "combine_subset:", com_index
    #train_dummy_top = combine_selection(train_dummy)
    #train_dummy = pd.get_dummies(train_combine)
    total_score = []
    total_time = []
    (sorted_dic, whole_train_dummy) = combine_selection(train_combine.iloc[train_val_index],val_label[train_val_index])
    test_dummy = pd.get_dummies(train_combine.iloc[test_val_index])
    for threshold in xrange(100,520,20):
        print "top: ", threshold
        val_time = []
        now_score = []
        for train_val_index, test_val_index in skf:    

            lda = LDA()
            sorted_feature = [i[0] for i in sorted_dic][:threshold]
            train_dummy_top = whole_train_dummy[sorted_feature]
            test_dummy_top = pd.DataFrame(np.zeros([test_dummy.shape[0], len(sorted_feature)]),columns=sorted_feature, index=test_dummy.index)
            

            cross_column = list(set(sorted_feature) & set(test_dummy.columns))
            test_dummy[cross_column] = test_dummy[cross_column]
            #print sorted_dic
            try: 
                train_lda = lda.fit_transform(train_dummy_top, val_label[train_val_index])        
                clf.fit(train_lda, val_label[train_val_index])
                test_lda = lda.transform(test_dummy_top)
                start = time.time()
                now_score.append(accuracy_score(clf.predict(test_lda), val_label[test_val_index]))
                val_time.append(time.time() - start)   
            
            except LinAlgError:
                None
        exe_time = np.sum(val_time)
        #total_score.append(np.average(now_score))
        #total_time.append(exe_time)

        print "combine_num", combine_num
        print "score:", np.average(now_score)
        print "time:", exe_time
        result.loc[exp_num] = [combine_num, threshold, np.average(now_score), exe_time]
        exp_num = exp_num + 1
        result.to_csv('dna_combine_lda_normal.csv')

In [185]:
# Do not run yet
train=pd.read_csv('splice.data.txt', header=None)
train.columns=['label', 'name', 'dna']
key_str = {}
for (i,label_name) in zip(range(len(np.unique(train['label']))), np.unique(train['label'])):
    key_str[label_name] = i
    
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key == label else None,key_str.keys()), train['label'])
train_dummy = []
    
train['dna'] = train['dna'].map(lambda x: list(str(x).strip()))
for idx in xrange(60):
    train['dna_%d'% (idx,)] = train['dna'].map(lambda x: x[idx])

train_column = train.columns[3:]
train = train[train_column]

result = pd.DataFrame(columns=['combine_num','top','score', 'time'])


total_score = []
total_time = []
clf = SVC(kernel='linear', C=1.0)
#clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)

exp_num = 0
for combine_num in range(2,4):
    
    train_combine = pd.DataFrame(index=range(len(train)))
    com_index = 0
    for sub in Cb(train.columns,combine_num):
        combine_list = []
        map(lambda data: combine_list.append(combine_function(data)),train[list(sub)].values)
        train_combine['combine%d' % com_index] = combine_list
        com_index = com_index + 1
    print "combine_subset:", com_index
    #train_dummy_top = combine_selection(train_dummy)
    #train_dummy = pd.get_dummies(train_combine)
    total_score = []
    total_time = []
    for threshold in xrange(100,520,20):
        print "top: ", threshold
        val_time = []
        now_score = []
        cv= 0 
        for train_val_index, test_val_index in skf:    
            print "cv:", cv+1
            cv = cv+1
            (train_dummy_top, test_dummy_top) = combine_selection(train_combine.iloc[train_val_index],val_label[train_val_index], train_combine.iloc[test_val_index], threshold = threshold)
            lda = LDA()
            try: 
                train_lda = lda.fit_transform(train_dummy_top, val_label[train_val_index])        
                clf.fit(train_lda, val_label[train_val_index])
                test_lda = lda.transform(test_dummy_top)
                start = time.time()
                now_score.append(accuracy_score(clf.predict(test_lda), val_label[test_val_index]))
                val_time.append(time.time() - start)   
            
            except LinAlgError:
                None
        exe_time = np.sum(val_time)
        #total_score.append(np.average(now_score))
        #total_time.append(exe_time)

        print "combine_num", combine_num
        print "score:", np.average(now_score)
        print "time:", exe_time
        result.loc[exp_num] = [combine_num, threshold, np.average(now_score), exe_time]

        result.to_csv('dna_combine_lda_normal.csv')

combine_subset: 1770
top:  100
cv: 1
Information gain process time: 427.385839939
Feature selection process time: 3.09240698814
cv: 2
Information gain process time: 429.710958958
Feature selection process time: 3.23030805588
cv: 3
Information gain process time: 425.141069889
Feature selection process time: 2.90448904037
cv: 4
Information gain process time: 437.889420986
Feature selection process time: 2.99455285072
cv: 5
Information gain process time: 425.141784191
Feature selection process time: 2.9996650219
cv: 6
Information gain process time: 429.429824114
Feature selection process time: 3.00812411308
cv: 7
Information gain process time: 423.960889101
Feature selection process time: 2.91519188881
cv: 8
Information gain process time: 431.128167152
Feature selection process time: 3.12298321724
cv: 9
Information gain process time: 433.847805977
Feature selection process time: 3.17117619514
cv: 10
Information gain process time: 425.238775969
Feature selection process time: 3.08852696419

KeyboardInterrupt: 

In [192]:
skf.

TypeError: 'NoneType' object is not callable

In [195]:
for num,(_,b) in zip(range(10),skf):
    print num, len(b)

0 320
1 320
2 320
3 320
4 320
5 319
6 319
7 318
8 317
9 317


# Mushroom

In [171]:
mush_columns = ['label', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size',\
               'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring',\
               'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type',\
               'spore-print-color', 'population', 'habitat']
train = pd.read_csv('agaricus-lepiota.data',header=None)
train.columns = mush_columns
key_str = {'e':0, 'p':1}
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key == label else None,key_str.keys()), train['label'])
del train['label']

result = pd.DataFrame(columns=['combine_num','top','score', 'time'])


total_score = []
total_time = []
clf = SVC(kernel='linear', C=1.0)
#clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)

exp_num = 0
for combine_num in range(2,4):
    
    train_combine = pd.DataFrame(index=range(len(train)))
    com_index = 0
    for sub in Cb(train.columns,combine_num):
        combine_list = []
        map(lambda data: combine_list.append(combine_function(data)),train[list(sub)].values)
        train_combine['combine%d' % com_index] = combine_list
        com_index = com_index + 1
    print "combine_subset:", com_index
    #train_dummy_top = combine_selection(train_dummy)
    #train_dummy = pd.get_dummies(train_combine)
    total_score = []
    total_time = []
    for threshold in xrange(100,520,20):
        print "top: ", threshold
        val_time = []
        now_score = []
        for train_val_index, test_val_index in skf:    

            (train_dummy_top, test_dummy_top) = combine_selection(train_combine.iloc[train_val_index],val_label[train_val_index], train_combine.iloc[test_val_index], threshold = threshold)
            lda = LDA()
            try: 
                train_lda = lda.fit_transform(train_dummy_top, val_label[train_val_index])        
                clf.fit(train_lda, val_label[train_val_index])
                test_lda = lda.transform(test_dummy_top)
                start = time.time()
                now_score.append(accuracy_score(clf.predict(test_lda), val_label[test_val_index]))
                val_time.append(time.time() - start)   
            
            except LinAlgError:
                None
        exe_time = np.sum(val_time)
        #total_score.append(np.average(now_score))
        #total_time.append(exe_time)

        print "combine_num", combine_num
        print "score:", np.average(now_score)
        print "time:", exe_time
        result.loc[exp_num] = [combine_num, threshold, np.average(now_score), exe_time]
        exp_num = exp_num + 1
        result.to_csv('mush_combine_lda_normal.csv')

combine_subset: 231
top:  100
combine_num 2
score: 0.983243016154
time: 0.0170269012451
top:  120
combine_num 2
score: 0.931156166062
time: 0.0170269012451
top:  140
combine_num 2
score: 0.945170722977
time: 0.0170269012451
top:  160
combine_num 2
score: 0.930658094497
time: 0.0170269012451
top:  180
combine_num 2
score: 0.927955707424
time: 0.0170269012451
top:  200
combine_num 2
score: 0.927091665491
time: 0.0170269012451
top:  220
combine_num 2
score: 0.949515502553
time: 0.0170269012451
top:  240
combine_num 2
score: 0.934113352231
time: 0.0170269012451
top:  260
combine_num 2
score: 0.93029485412
time: 0.0170269012451
top:  280
combine_num 2
score: 0.946291383999
time: 0.0170269012451
top:  300
combine_num 2
score: 0.952114604993
time: 0.0170269012451
top:  320
combine_num 2
score: 0.941509502641
time: 0.0170269012451
top:  340
combine_num 2
score: 0.954775132215
time: 0.0170269012451
top:  360
combine_num 2
score: 0.944099201781
time: 0.0170269012451
top:  380
combine_num 2
score

KeyboardInterrupt: 

In [7]:
mush_columns = ['label', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size',\
               'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring',\
               'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type',\
               'spore-print-color', 'population', 'habitat']
train = pd.read_csv('agaricus-lepiota.data',header=None)
train.columns = mush_columns
key_str = {'e':0, 'p':1}
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key == label else None,key_str.keys()), train['label'])
del train['label']

result = pd.DataFrame(columns=['combine_num','c_cluster','score', 'time'])


total_score = []
total_time = []
#clf = SVC(kernel='linear', C=1.0)
clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)

exp_num =0 
for combine_num in range(2,3):
    
    train_combine = pd.DataFrame(index=range(len(train)))
    com_index = 0
    for sub in Cb(train.columns,combine_num):
        combine_list = []
        map(lambda data: combine_list.append(combine_function(data)),train[list(sub)].values)
        train_combine['combine_%d' % com_index] = combine_list
        com_index = com_index + 1
    print "combine_subset:", com_index
    train_dummy = pd.get_dummies(train_combine)
    for cluster in range(2,4):
        total_score = []
        total_time = []
        for iteration in range(5):
            print iteration
            val_time = []
            now_score = []
            for train_val_index, test_val_index in skf:    
                lda = LDA()
                try:    
                    train_lda = lda.fit_transform(train_dummy.iloc[train_val_index], val_label[train_val_index])
                    test_lda = lda.transform(train_dummy.iloc[test_val_index])
                    (train_lda, test_lda) = normalization(train_lda, test_lda)
                    clf.fit(train_lda, val_label[train_val_index])
                    
                    start = time.time()
                    now_score.append(accuracy_score(clf.predict(test_lda), val_label[test_val_index]))
                    val_time.append(time.time() - start)   
                except LinAlgError:
                    None
            exe_time = np.sum(val_time)
            total_score.append(np.average(now_score))
            total_time.append(exe_time)
            
        print "combine_num", combine_num
        print "n_cluster:", cluster
        print "score:", np.average(total_score)
        print "time:", np.average(total_time)
        result.loc[exp_num] = [combine_num, cluster, np.average(total_score), np.average(total_time)]
        exp_num = exp_num + 1
        result.to_csv('mush_combine_lda_normal.csv')

combine_subset: 231
0
1
2
3
4
combine_num 2
n_cluster: 2
score: 0.519078531668
time: 0.0060375213623
0
1
2
3
4
combine_num 2
n_cluster: 3
score: 0.519078531668
time: 0.00495452880859


# Car

In [196]:
car_columns = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'label']
train = pd.read_csv('car.data',header=None)
train.columns = car_columns
key_str = {'unacc':0, 'acc':1, 'good':2, 'vgood':3}
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key == label else None,key_str.keys()), train['label'])
del train['label']

result = pd.DataFrame(columns=['combine_num','c_cluster','score', 'time'])



total_score = []
total_time = []
clf = SVC(kernel='linear', C=1.0)
#clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)

exp_num = 0
highest_score = 0
highest_idx = 0
for combine_num in range(2,5):
    
    train_combine = pd.DataFrame(index=range(len(train)))
    com_index = 0
    for sub in Cb(train.columns,combine_num):
        combine_list = []
        map(lambda data: combine_list.append(combine_function(data)),train[list(sub)].values)
        train_combine['combine%d' % com_index] = combine_list
        com_index = com_index + 1
    print "combine_subset:", com_index
    #train_dummy_top = combine_selection(train_dummy)
    #train_dummy = pd.get_dummies(train_combine)
    total_score = []
    total_time = []
    for threshold in xrange(100,520,20):
        print "top: ", threshold
        val_time = []
        now_score = []
        for train_val_index, test_val_index in skf:    

            (train_dummy_top, test_dummy_top) = combine_selection(train_combine.iloc[train_val_index],val_label[train_val_index], train_combine.iloc[test_val_index], threshold = threshold)
            lda = LDA()
            try: 
                train_lda = lda.fit_transform(train_dummy_top, val_label[train_val_index])        
                clf.fit(train_lda, val_label[train_val_index])
                test_lda = lda.transform(test_dummy_top)
                start = time.time()
                now_score.append(accuracy_score(clf.predict(test_lda), val_label[test_val_index]))
                val_time.append(time.time() - start)   
            
            except LinAlgError:
                None
        exe_time = np.sum(val_time)
        #total_score.append(np.average(now_score))
        #total_time.append(exe_time)
        
        if np.average(now_score) > highest_score:
            highest_score = np.average(now_score)
            highest_idx = exp_num
        
        print "combine_num", combine_num, "score:", np.average(now_score), "time:", exe_time
        
        result.loc[exp_num] = [combine_num, threshold, np.average(now_score), exe_time]
        exp_num = exp_num + 1
        #result.to_csv('car_combine_lda_normal.csv')
print "Highest:", result.loc[highest_idx]

combine_subset: 15
top:  100
Information gain process time: 0.127707958221
Feature selection process time: 0.0605630874634
Information gain process time: 0.0708389282227
Feature selection process time: 0.0496971607208
Information gain process time: 0.0701961517334
Feature selection process time: 0.0544741153717
Information gain process time: 0.070013999939
Feature selection process time: 0.0518810749054
Information gain process time: 0.0708959102631
Feature selection process time: 0.0513870716095
Information gain process time: 0.0696289539337
Feature selection process time: 0.0540490150452
Information gain process time: 0.0718469619751
Feature selection process time: 0.0490601062775
Information gain process time: 0.0704591274261
Feature selection process time: 0.0522990226746
Information gain process time: 0.0699801445007
Feature selection process time: 0.0527260303497
Information gain process time: 0.0706050395966
Feature selection process time: 0.0491740703583
combine_num 2 score: 0.

KeyboardInterrupt: 

In [3]:
car_columns = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'label']
train = pd.read_csv('car.data',header=None)
train.columns = car_columns
key_str = {'unacc':0, 'acc':1, 'good':2, 'vgood':3}
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key == label else None,key_str.keys()), train['label'])
del train['label']

result = pd.DataFrame(columns=['combine_num','c_cluster','score', 'time'])



total_score = []
total_time = []
#clf = SVC(kernel='linear', C=1.0)
clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)

exp_num = 0
for combine_num in range(2,5):
    
    train_combine = pd.DataFrame(index=range(len(train)))
    com_index = 0
    for sub in Cb(train.columns,combine_num):
        combine_list = []
        map(lambda data: combine_list.append(combine_function(data)),train[list(sub)].values)
        train_combine['combine%d' % com_index] = combine_list
        com_index = com_index + 1
    #print "combine_subset:", com_index
    train_dummy = pd.get_dummies(train_combine)
    for cluster in range(2,5):
        total_score = []
        total_time = []
        for iteration in range(10):
            val_time = []
            now_score = []
            for train_val_index, test_val_index in skf:    
                lda = LDA()
                train_lda = lda.fit_transform(train_dummy.iloc[train_val_index], val_label[train_val_index])        
                clf.fit(train_lda, val_label[train_val_index])
                test_lda = lda.transform(train_dummy.iloc[test_val_index])
                start = time.time()
                now_score.append(accuracy_score(clf.predict(test_lda), val_label[test_val_index]))
                val_time.append(time.time() - start)         
            exe_time = np.sum(val_time)
            total_score.append(np.average(now_score))
            total_time.append(exe_time)
            
        print "combine_num", combine_num
        print "n_cluster:", cluster
        print "score:", np.average(total_score)
        print "time:", np.average(total_time)
        result.loc[exp_num] = [combine_num, cluster, np.average(total_score), np.average(total_time)]
        exp_num = exp_num + 1
        result.to_csv('car_combine_lda_normal.csv')

/usr/anaconda2/lib/python2.7/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


combine_num 2
n_cluster: 2
score: 0.693977698049
time: 0.0135494947433
combine_num 2
n_cluster: 3
score: 0.693977698049
time: 0.00973460674286
combine_num 2
n_cluster: 4
score: 0.693977698049
time: 0.0112968921661
combine_num 3
n_cluster: 2
score: 0.691818751238
time: 0.00888221263885
combine_num 3
n_cluster: 3
score: 0.691818751238
time: 0.00874812602997
combine_num 3
n_cluster: 4
score: 0.691818751238
time: 0.00826539993286
combine_num 4
n_cluster: 2
score: 0.675885464194
time: 0.00815374851227
combine_num 4
n_cluster: 3
score: 0.675885464194
time: 0.00594704151154
combine_num 4
n_cluster: 4
score: 0.675885464194
time: 0.00691163539886


# Bot

In [4]:
column_str = ['Proto', 'State','Dir','Dport']
column_numeric = ['Dur', 'sTos', 'dTos', 'TotPkts', 'TotBytes','SrcBytes']
train = pd.read_csv('botnet_5.binetflow')

key_str = {'Normal':0, 'Botnet':1, 'Background':2}
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key in label else None,key_str.keys()), train['Label'])

train_index = np.array(np.where(np.array(label_num) == 0)[0].tolist() + np.where(np.array(label_num) == 1)[0].tolist())
train = train.iloc[train_index]
train = train.fillna(0)
label_num = np.array(label_num)[train_index]
train_numeric = train[column_numeric]
train = train[column_str]

result = pd.DataFrame(columns=['combine_num','c_cluster','score', 'time'])



total_score = []
total_time = []
#clf = SVC(kernel='linear', C=1.0)
clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)

exp_num=0
for combine_num in range(2,4):
    
    train_combine = pd.DataFrame(index=range(len(train)))
    com_index = 0
    for sub in Cb(train.columns,combine_num):
        combine_list = []
        map(lambda data: combine_list.append(combine_function(data)),train[list(sub)].values)
        train_combine['combine_%d' % com_index] = combine_list
        com_index = com_index + 1
    #print "combine_subset:", com_index
    train_dummy = pd.get_dummies(train_combine)
    for cluster in range(2,4):
        total_score = []
        total_time = []
        for iteration in range(10):
            val_time = []
            now_score = []
            for train_val_index, test_val_index in skf:  
                lda = LDA()
                train_lda = lda.fit_transform(train_dummy.iloc[train_val_index], val_label[train_val_index]) 
                train_lda = np.append(train_lda, train_numeric.iloc[train_val_index].values, axis=1)
                test_lda = lda.transform(train_dummy.iloc[test_val_index])
                test_lda = np.append(test_lda, train_numeric.iloc[test_val_index].values, axis=1)        
                (train_lda, test_lda) = normalization(train_lda, test_lda)

                clf.fit(train_lda, val_label[train_val_index])
                start = time.time()
                now_score.append(f1_score(clf.predict(test_lda), val_label[test_val_index]))
                val_time.append(time.time() - start)         
            exe_time = np.sum(val_time)
            total_score.append(np.average(now_score))
            total_time.append(exe_time)
            
        print "combine_num", combine_num
        print "n_cluster:", cluster
        print "score:", np.average(total_score)
        print "time:", np.average(total_time)
        result.loc[exp_num] = [combine_num, cluster, np.average(total_score), np.average(total_time)]
        exp_num = exp_num + 1
        result.to_csv('bot5_combine_lda_normal.csv')

/usr/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


combine_num 2
n_cluster: 2
score: 0.00864941510103
time: 0.011870598793
combine_num 2
n_cluster: 3
score: 0.00864941510103
time: 0.0109653234482
combine_num 3
n_cluster: 2
score: 0.00864941510103
time: 0.0115445613861
combine_num 3
n_cluster: 3
score: 0.00864941510103
time: 0.0123892307281


In [5]:
column_str = ['Proto', 'State','Dir','Dport']
column_numeric = ['Dur', 'sTos', 'dTos', 'TotPkts', 'TotBytes','SrcBytes']
train = pd.read_csv('botnet_6.binetflow')

key_str = {'Normal':0, 'Botnet':1, 'Background':2}
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key in label else None,key_str.keys()), train['Label'])

train_index = np.array(np.where(np.array(label_num) == 0)[0].tolist() + np.where(np.array(label_num) == 1)[0].tolist())
train = train.iloc[train_index]
train = train.fillna(0)
label_num = np.array(label_num)[train_index]
train_numeric = train[column_numeric]
train = train[column_str]

result = pd.DataFrame(columns=['combine_num','c_cluster','score', 'time'])



total_score = []
total_time = []
#clf = SVC(kernel='linear', C=1.0)
clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)

exp_num=0
for combine_num in range(2,4):
    
    train_combine = pd.DataFrame(index=range(len(train)))
    com_index = 0
    for sub in Cb(train.columns,combine_num):
        combine_list = []
        map(lambda data: combine_list.append(combine_function(data)),train[list(sub)].values)
        train_combine['combine_%d' % com_index] = combine_list
        com_index = com_index + 1
    #print "combine_subset:", com_index
    train_dummy = pd.get_dummies(train_combine)
    for cluster in range(2,4):
        total_score = []
        total_time = []
        for iteration in range(10):
            val_time = []
            now_score = []
            for train_val_index, test_val_index in skf:  
                lda = LDA()
                train_lda = lda.fit_transform(train_dummy.iloc[train_val_index], val_label[train_val_index]) 
                train_lda = np.append(train_lda, train_numeric.iloc[train_val_index].values, axis=1)
                test_lda = lda.transform(train_dummy.iloc[test_val_index])
                test_lda = np.append(test_lda, train_numeric.iloc[test_val_index].values, axis=1)        
                (train_lda, test_lda) = normalization(train_lda, test_lda)

                clf.fit(train_lda, val_label[train_val_index])
                start = time.time()
                now_score.append(f1_score(clf.predict(test_lda), val_label[test_val_index]))
                val_time.append(time.time() - start)         
            exe_time = np.sum(val_time)
            total_score.append(np.average(now_score))
            total_time.append(exe_time)
            
        print "combine_num", combine_num
        print "n_cluster:", cluster
        print "score:", np.average(total_score)
        print "time:", np.average(total_time)
        result.loc[exp_num] = [combine_num, cluster, np.average(total_score), np.average(total_time)]
        exp_num = exp_num + 1
        result.to_csv('bot6_combine_lda_normal.csv')

combine_num 2
n_cluster: 2
score: 0.54816706972
time: 0.0186123132706
combine_num 2
n_cluster: 3
score: 0.54816706972
time: 0.0222583293915
combine_num 3
n_cluster: 2
score: 0.54816706972
time: 0.0243029356003
combine_num 3
n_cluster: 3
score: 0.54816706972
time: 0.021257853508


In [6]:
column_str = ['Proto', 'State','Dir','Dport']
column_numeric = ['Dur', 'sTos', 'dTos', 'TotPkts', 'TotBytes','SrcBytes']
train = pd.read_csv('botnet_7.binetflow')

key_str = {'Normal':0, 'Botnet':1, 'Background':2}
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key in label else None,key_str.keys()), train['Label'])

train_index = np.array(np.where(np.array(label_num) == 0)[0].tolist() + np.where(np.array(label_num) == 1)[0].tolist())
train = train.iloc[train_index]
train = train.fillna(0)
label_num = np.array(label_num)[train_index]
train_numeric = train[column_numeric]
train = train[column_str]

result = pd.DataFrame(columns=['combine_num','c_cluster','score', 'time'])



total_score = []
total_time = []
#clf = SVC(kernel='linear', C=1.0)
clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)

exp_num=0
for combine_num in range(2,4):
    
    train_combine = pd.DataFrame(index=range(len(train)))
    com_index = 0
    for sub in Cb(train.columns,combine_num):
        combine_list = []
        map(lambda data: combine_list.append(combine_function(data)),train[list(sub)].values)
        train_combine['combine_%d' % com_index] = combine_list
        com_index = com_index + 1
    #print "combine_subset:", com_index
    train_dummy = pd.get_dummies(train_combine)
    for cluster in range(2,4):
        total_score = []
        total_time = []
        for iteration in range(10):
            val_time = []
            now_score = []
            for train_val_index, test_val_index in skf:  
                lda = LDA()
                train_lda = lda.fit_transform(train_dummy.iloc[train_val_index], val_label[train_val_index]) 
                train_lda = np.append(train_lda, train_numeric.iloc[train_val_index].values, axis=1)
                test_lda = lda.transform(train_dummy.iloc[test_val_index])
                test_lda = np.append(test_lda, train_numeric.iloc[test_val_index].values, axis=1)        
                (train_lda, test_lda) = normalization(train_lda, test_lda)

                clf.fit(train_lda, val_label[train_val_index])
                start = time.time()
                now_score.append(f1_score(clf.predict(test_lda), val_label[test_val_index]))
                val_time.append(time.time() - start)         
            exe_time = np.sum(val_time)
            total_score.append(np.average(now_score))
            total_time.append(exe_time)
            
        print "combine_num", combine_num
        print "n_cluster:", cluster
        print "score:", np.average(total_score)
        print "time:", np.average(total_time)
        result.loc[exp_num] = [combine_num, cluster, np.average(total_score), np.average(total_time)]
        exp_num = exp_num + 1
        result.to_csv('bot7_combine_lda_normal.csv')

combine_num 2
n_cluster: 2
score: 0.025
time: 0.010089468956
combine_num 2
n_cluster: 3
score: 0.025
time: 0.0104130268097
combine_num 3
n_cluster: 2
score: 0.025
time: 0.0105338096619
combine_num 3
n_cluster: 3
score: 0.025
time: 0.0101867198944
